# 2. Data Preparation

2장에 대한 수정 작업을 develop 브랜치에서 진행하고 있음. 계속 진행하는 중. 한번 더 수정

## 2.1 Travel Demand Data (Trip record data)

수도권 생활이동 데이터 가공

In [2]:
a = [1,2,3] 

## 2.2 Road Network and Geometry

### 2.2.1 Open Street Map

OSM을 이용해서 수도권 지역의 도로 데이터를 가져오고, 시각화 하는 작업을 수행

## 2.3 GTFS (General Transit Feed Specification)